In [1]:
from grafeno import Graph

In [2]:
from grafeno.transformers import get_pipeline

In [3]:
import yaml
semantic_pipeline = yaml.load(open('configs/semantic.yaml'))
T = get_pipeline(['spacy_parse']+semantic_pipeline.get('transformers'))

In [4]:
text = """
An extreme goal of AI research is to create computer programs that can learn, solve problems, and think logically.
In practice, however, most applications have picked on problems which computers can do well.
Searching data bases and doing calculations are things computers do better than people.
On the other hand, "perceiving its environment" in any real sense is way beyond present-day computing.
"""

text = """
John loves Mary. She loves John.
"""

In [5]:
G = Graph(text=text, transformer=T)

In [6]:
from grafeno.jupyter import visualize

In [7]:
visualize(G)